# Day 3 - Overlapping rectangles

* [Day 3](https://adventofcode.com/2018/day/3)

This is a computational geometry problem; we can use a [sweep line algorithm](https://en.wikipedia.org/wiki/Sweep_line_algorithm) to reduce the problem from 2 to just 1 dimension, moving a line across the board to focus on overlapping *intervals*.

In [1]:
import re
from dataclasses import dataclass, field
from typing import Sequence

_parse_line = re.compile(r'#(\d+)\s*@\s*(\d+),(\d+):\s*(\d+)x(\d+)').search
ID = int  # type alias
                  
@dataclass
class Rectangle:
    id: ID
    left: int
    right: int = field(init=False)
    top: int
    bottom: int = field(init=False)
    width: int
    height: int
    xrange: range = field(init=False)
    yrange: range = field(init=False)
    
    def __post_init__(self) -> None:
        self.right = self.left + self.width
        self.bottom = self.top + self.height
        self.xrange = range(self.left, self.right)
        self.yrange = range(self.top, self.bottom)
    
    @classmethod
    def from_line(cls, line: str) -> 'Rectangle':
        match = _parse_line(line)
        assert match is not None
        id, left, top, width, height = map(int, match.groups())
        return cls(id, left, top, width, height)

    @classmethod
    def all_from_lines(cls, lines: str) -> Sequence['Rectangle']:
        return sorted(map(cls.from_line, lines.splitlines()), key=attrgetter("top"))

In [2]:
testrect = Rectangle.from_line("#123 @ 3,2: 5x4\n")
assert (testrect.left, testrect.top, testrect.width, testrect.height) == (3, 2, 5, 4)
assert testrect.right, testrect.bottom == (8, 6)
assert testrect.xrange == range(3, 8)
assert testrect.yrange == range(2, 6)

This is basically an interval merging problem.

* At each line `x`, find all the rectangles that intersect that line, as intervals (top, bottom)
* Count how many `y` coordinates overlap.
* Take into account that 1 large interval could overlap multiple following intervals that themselves do not overlap.

You can do this by taking the minimum of (furthest bottom, current bottom) minus the current top value, provided top < previous bottom edge:

```
 0
 1 top
 2  |
 3  |  top  X  5 - 3 == 2
 4  |   |   X  5 is the min(furthest bottom, bot) value here
 5 bot  |
 6      |
 7     bot
 8 
 9 top
10  |
11  |  top X  13 - 11 == 2
12  |   |  X  13 is the min(furthest bottom, bot) value here
13  |  bot
14  |
15  |  top X  17 - 15 == 2
16  |   |  X  17 is the min(furthest bottom, bot) value here
17  |  bot
18 bot
```

 Gotcha is multiple rectangles overlapping on the same `y` coordinates overlap:

```
 0
 1 top
 2  |
 3  |  top     X
 4  |   |  top X <- only one overlapping square, don't count this twice
 5 bot  |   |  X
 6      |   |  X total 4
 7     bot  |
 8          |
 9         bot
```

So we track the furthest overlap already handled by storing the previous bottom value, to prefer over the current top value. In the above, we already handled the (5 - 3) overlap, so the next overlap to calculate should use 5 rather than 4 for the `top` value.

Another edgecase is where a short interval follows an already overlapping longer interval; the bottom of the shorter interval is then above the already covered overlap bottom:

```
 0
 1 top
 2  |
 3  |  top     X
 4  |   |  top X The short interval can be ignored here
 5  |   |   |  X
 6  |   |  bot X
 7  |   |      X total 5  
 8 bot  |       
 9     bot     
```

Because the rectangle sizes are small even in the puzzle dataset, it's simplest to do this with `range()` objects and `set` intersections; a previous iteration calculated the differences between tops and bottoms, which also works.

In [3]:
from operator import attrgetter, methodcaller
from typing import Set

def sum_overlaps(rectangles: Sequence[Rectangle]) -> int:
    max_x = max(r.right for r in rectangles)
    total = 0
    for x in range(max_x):
        overlap: Set[int] = set()
        # all y ranges for rectangles on the current line
        intervals = (r.yrange for r in rectangles if x in r.xrange)
        try:
            furthest = next(intervals)
        except StopIteration:
            # no matching rectangles at this x position
            continue
        for r in intervals:
            if r.start < furthest.stop:
                overlap |= set(furthest).intersection(r)
            if r.stop > furthest.stop:
                furthest = r
        total += len(overlap)
    return total

In [4]:
test1 = Rectangle.all_from_lines("""\
#1 @ 1,3: 4x4
#2 @ 3,1: 4x4
#3 @ 5,5: 2x2
""")
assert sum_overlaps(test1) == 4
test2 = Rectangle.all_from_lines("""\
#1 @ 1,9: 1x9
#2 @ 1,11: 1x2
#3 @ 1,15: 1x2
""")
assert sum_overlaps(test2) == 4
test3 = Rectangle.all_from_lines("""\
#1 @ 1,1: 1x4
#2 @ 1,3: 1x4
#3 @ 1,4: 1x5
""")
assert sum_overlaps(test3) == 4
test4 = Rectangle.all_from_lines("""\
#1 @ 1,1: 1x7
#2 @ 1,3: 1x6
#3 @ 1,4: 1x2
""")
assert sum_overlaps(test4) == 5

In [5]:
import aocd
data = aocd.get_data(day=3, year=2018)
rectangles = Rectangle.all_from_lines(data)

In [6]:
print("Part 1:", sum_overlaps(rectangles))

Part 1: 106501


## Part 2 - Finding the single rectangle that doesn't overlap

For this, we can just use a set containing all ids, and remove the ids of rectangles involved in intersections. In the end only a single id remains.

In [7]:
def no_overlaps(rectangles: Sequence[Rectangle]) -> ID:
    max_x = max(r.right for r in rectangles)
    no_overlaps = set(map(attrgetter('id'), rectangles))
    for x in range(max_x):
        active_rects = (rect for rect in rectangles if x in rect.xrange)
        try:
            furthest = next(active_rects)
        except StopIteration:
            # No matching rectangles at this x position
            continue
        for rect in active_rects:
            if rect.top < furthest.bottom:
                no_overlaps.difference_update((rect.id, furthest.id))
            if rect.bottom > furthest.bottom:
                furthest = rect
    remaining, = no_overlaps  # only works if there is exactly 1 element
    return remaining

In [8]:
assert no_overlaps(test1) == 3

In [9]:
print("Part 2:", no_overlaps(rectangles))

Part 2: 632
